### Import Libraries

In [ ]:
# Importing necessary libraries
import mysql.connector  # For MySQL database connection
import pandas as pd  # For data manipulation and analysis
import matplotlib.pyplot as plt  # For data visualization
import seaborn as sns  # For advanced visualizations
import warnings  # To manage warnings in code

# Suppress warnings to improve readability of output
warnings.filterwarnings('ignore')

### Connecting to Database

In [ ]:
# Establishing a connection to the MySQL database
connection = mysql.connector.connect(host = '@@@',
                                     user = '@@@',
                                     password = '@@@',
                                     database = '@@@')

# Running an SQL query to retrieve all the table names from the connected database
tech_tables = pd.read_sql_query('SHOW TABLES', connection)

# Displaying the list of tables in the 'techtrendpro' database
tech_tables

### Extracting Data from SQL

In [ ]:
# Loading data from the 'feedback_data' table in the database
feedback = pd.read_sql_query('SELECT * FROM feedback_data', connection)
feedback.head()  # Displaying the first few rows of the feedback data

In [ ]:
# Loading data from the 'product_data' table in the database
product = pd.read_sql_query('SELECT * FROM product_data', connection)
product.head()  # Displaying the first few rows of the product data

In [ ]:
# Loading data from the 'sales_data' table in the database
sales = pd.read_sql_query('SELECT * FROM sales_data', connection)
sales.head()  # Displaying the first few rows of the sales data

### Inspecting Headers

In [ ]:
# Retrieving column information from the 'feedback_data' table
feedback_headers = pd.read_sql_query('SHOW COLUMNS FROM feedback_data', connection)
feedback_headers  # Display column details

In [ ]:
# Retrieving column information from the 'product_data' table
product_headers = pd.read_sql_query('SHOW COLUMNS FROM product_data', connection)
product_headers  # Display column details

In [ ]:
# Retrieving column information from the 'sales_data' table
sales_headers = pd.read_sql_query('SHOW COLUMNS FROM sales_data', connection)
sales_headers  # Display column details

### Descriptive Analysis

In [ ]:
# Generating descriptive statistics for the feedback data
feedback_stats = feedback.describe(include='all')
feedback_stats  # Display the statistics

In [ ]:
# Generating descriptive statistics for the product data
product_stats = product.describe(include='all')
product_stats  # Display the statistics

In [ ]:
# Generating descriptive statistics for the sales data
sales_stats = sales.describe(include='all')
sales_stats  # Display the statistics

### Joining Data

In [ ]:
# Aggregating sales data by product to sum volumes and revenues
sales_aggregated = sales.groupby('productid').agg({
    'saleid': 'first',  # Retaining first sale ID for each product
    'salesdate': 'first',  # Retaining first sale date for each product
    'salesvolume': 'sum',  # Summing sales volume
    'revenuegenerated': 'sum'  # Summing revenue
}).reset_index()

# Merging the aggregated sales data with the product data
salesproduct_merged = pd.merge(sales_aggregated, product, on='productid', how='inner')

# Merging feedback data with the sales and product data
all_data = pd.merge(feedback, salesproduct_merged, on='productid', how='inner')

In [ ]:
# Exporting the merged data to a CSV file for further analysis or record-keeping
all_data.to_csv('joined_data.csv', index=False)

In [ ]:
all_data.head(10)

In [ ]:
all_data.info()

In [ ]:
# Calculating product return rate based on feedback count and total sales volume
productreturn_rate = (all_data['feedbackid'].count() * 100) / all_data['salesvolume'].sum()
print(f'Sales Volume: {all_data['salesvolume'].sum()}\nReturn Volume: {all_data['feedbackid'].count()}\nReturn Rate: {round(productreturn_rate, 2)}%')

### Exploratory Data Analysis (EDA) - Univariate

In [ ]:
# Visualizing sales volume, revenue, and other product statistics

plt.style.use('classic')
fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(12, 6), gridspec_kw={'hspace': 0.4})

# Custom color palette
custom_colors = ['#0072B2', '#E69F00', '#009E73', '#CC79A7']

# Plot distribution of Sales Volume
sns.histplot(all_data['salesvolume'], kde=True, ax=axes[0, 0], color=custom_colors[3])
axes[0, 0].set(title='Distribution of Sales Volume', xlabel='Sales Volume', ylabel='Frequency')

# Plot distribution of Revenue Generated
sns.histplot(all_data['revenuegenerated'], kde=True, ax=axes[0, 1], color=custom_colors[2])
axes[0, 1].set(title='Distribution of Revenue Generated', xlabel='Revenue Generated', ylabel='Frequency')

# Sales Volume by Product Category
category_sales = all_data.groupby('productcategory')['salesvolume'].sum()
category_sales.plot(kind='bar', ax=axes[1, 0], color=custom_colors[1])
axes[1, 0].set(title='Sales Volume by Product Category', xlabel='Product Category', ylabel='Sales')

# Return Reasons by Count
return_reason_counts = all_data['returnreason'].value_counts()
return_reason_counts.plot(kind='bar', ax=axes[1, 1], color=custom_colors[0])
axes[1, 1].set(title='Return Reasons', xlabel='Return Reason', ylabel='Count')

# Overall plot title and layout adjustments
fig.suptitle('Univariate Analysis', fontsize=16)
plt.tight_layout(rect=[0, 0, 1, 0.95])
plt.show()

### Exploratory Data Analysis (EDA) - Bivariate

In [ ]:
# Create a crosstab to count occurrences of each combination
stacked_data = pd.crosstab(all_data['returnreason'], all_data['productcategory'])

# Sort columns by the sum of each column in descending order
stacked_data = stacked_data[stacked_data.sum(axis=0).sort_values(ascending=False).index]

# Define your custom colors
custom_colors = ['#0072B2', '#009E73', '#CC79A7']

# Plotting the stacked bar chart
stacked_data.plot(kind='barh', stacked=True, figsize=(12, 6), color=custom_colors)

# Set title and labels
plt.title('Return Reason By Product Category')
plt.xlabel('Count of Returns')
plt.ylabel('Return Reason')

# Show the legend outside the plot
plt.legend(title='Product Category', bbox_to_anchor=(1.05, 1), loc='upper left')

# Adjust layout
plt.suptitle('Bivariate Analysis', fontsize=16)
plt.tight_layout(rect=[0, 0, 1, 0.95])
plt.show()

### Analysis

In [ ]:
# Extracting insights from 'Customer Feedback' related to product quality
# Assume feedback mentioning terms like 'defective', 'poor quality', 'not as described' etc. indicate quality issues
quality_related_terms = ['defective',
                        'poor quality',
                        'disappointing',
                        'poor',
                        'malfunctioning',
                        'faulty',
                        'subpar',
                        'unsatisfactory',
                        'unreliable',
                        'underwhelming',
                        'limited functionality',
                        'performance issues',
                        'software bugs',
                        'hardware failure',
                        'glitchy',
                        'inconsistent',
                        'slow',
                        'overheating',
                        'not worth the price',
                        'broke after a few uses',
                        'short lifespan',
                        'low battery life',
                        'not responsive',
                        'poor sound quality',
                        'screen issues',
                        'connectivity problems'
]


# Adding a new column 'Quality Issue' to indicate if the feedback suggests a quality issue
all_data['qualityissue'] = all_data['customerfeedback'].apply(
    lambda x: any(term in x.lower() for term in quality_related_terms)
)

# Analyzing the frequency of returns due to quality issues
quality_issue_returns = all_data['qualityissue'].value_counts()

# Visualizing the relationship between product category and quality issues
plt.figure(figsize=(12, 6))
custom_colors = ['#009E73', '#CC79A7']
sns.countplot(x='productcategory', hue='qualityissue', data=all_data, width=0.7, palette=custom_colors)

# Adjust layout
plt.title('Quality Issues by Product Category', fontsize=16)
plt.ylabel('Count of Quality Issues', fontsize=14)
plt.xlabel('Product Category', fontsize=14)
plt.legend(title='Quality Issues', bbox_to_anchor=(1.05, 1), loc='upper left')

plt.show()

In [ ]:
all_data.groupby('productcategory')['qualityissue'].value_counts()


In [ ]:
# To statistically ascertain the relationship between quality issues and return rates
from scipy.stats import chi2_contingency
import matplotlib.colors as mcolors

# Preparing a contingency table for the chi-square test
contingency_table = pd.crosstab(all_data['productcategory'], all_data['qualityissue'])

# Performing the chi-square test
chi2, p, dof, expected = chi2_contingency(contingency_table)

# Results of the chi-square test
chi2_test_result = {
    "Chi-Square Statistic": chi2,
    "p-value": p,
    "Degrees of Freedom": dof,
    "Expected Frequencies": expected
}

In [ ]:
quality_issue_returns

In [ ]:
chi2_test_result

## Visualisation

In [ ]:

# Preparing a contingency table for the chi-square test
contingency_table = pd.crosstab(all_data['productcategory'], all_data['qualityissue'])

# Performing the chi-square test
chi2, p, dof, expected = chi2_contingency(contingency_table)

# Results of the chi-square test
chi2_test_result = {
    "Chi-Square Statistic": chi2,
    "p-value": p,
    "Degrees of Freedom": dof,
    "Expected Frequencies": expected
}  

# Convert the expected frequencies to a DataFrame for easy plotting
expected_df = pd.DataFrame(expected,
                           index=contingency_table.index,
                           columns=contingency_table.columns)

# Plotting
fig, ax = plt.subplots(1, 2, figsize=(15, 10))

custom_cmap = mcolors.LinearSegmentedColormap.from_list("custom_cmap", ['#0072B2', '#E69F00', '#009E73', '#CC79A7'])

# Heatmap for Observed Frequencies
sns.heatmap(contingency_table, annot=True, annot_kws={"color": "black"}, cmap=custom_cmap, ax=ax[0], fmt='g')
ax[0].set_title('Observed Frequencies')
ax[0].set_xlabel('Quality Issues')
ax[0].set_ylabel('Product Category')

# Heatmap for Expected Frequencies
sns.heatmap(expected_df, annot=True, annot_kws={"color": "black"},cmap=custom_cmap, ax=ax[1], fmt='g')  
ax[1].set_title('Expected Frequencies')
ax[1].set_xlabel('Quality Issues')
ax[1].set_ylabel('Product Category')

plt.tight_layout()
plt.show()
